# Projeto 1 - Ciência dos Dados

Nome: Ananda Júlia Galvão Campelo

Nome: Carol Magario de Mattos Barretto

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [142]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import unidecode
from nltk import SnowballStemmer

In [143]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\Ananda\CDADOS 22.2\22-2a-cd-p1-grupo_anandajgc


Carregando a base de dados com as notícias classificadas manualmente:

In [144]:
filename = 'data/dados.xlsx'

In [145]:
train = pd.read_excel(filename, sheet_name = 'Treinamento')
train.head(5)

,Categoria,Titulo,PrimeiroParag,Data,Pagina,Target
0,Mercados,Ibovespa: ALPA4 fecha mais um pregão em alta e...,"O Ibovespa (IBOV) fechou em queda de 1,09% nes...",26/08/2022 - 17:37,43,2
1,Empresas,"IRB (IRBR3) é, de longe, a ação mais ‘shortead...",O IRB (IRBR3) é a ação com mais investidores s...,31/08/2022 - 17:00,28,1
2,Carteira Recomendada,5 ações que pagam bons dividendos para investi...,"Para setembro, a Órama Investimentos optou por...",05/09/2022 - 10:27,14,1
3,Carteira Recomendada,Empiricus Investimentos escala 6 ações com div...,"A Empiricus Investimentos, ex-Vitreo, incluiu ...",05/09/2022 - 13:21,12,1
4,Mercados,Ibovespa futuro opera em queda com aversão ao ...,O dólar abria o pregão desta segunda-feira (29...,29/08/2022 - 9:08,40,0


In [146]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Categoria,Titulo,PrimeiroParag,Data,Pagina,Target
0,Mercados,"Ibovespa (IBOV) vira para queda, com VALE3 ent...",O Ibovespa (IBOV) operava sem forças nesta qua...,24/08/2022 - 13:58,53.0,2
1,Empresas,Ação da Braskem (BRKM3) avança com notícia sob...,As ações da Braskem (BRKM3) avançavam ao redor...,02/09/2022 - 11:30,20.0,2
2,Mercados,"Compre commodities, se preocupe com recessão d...",O Goldman Sachs recomenda que os investidores ...,29/08/2022 - 9:49,40.0,1
3,Carteira Recomendada,Veja as 5 novas ações escaladas pela XP para e...,A XP Investimentos repaginou sua carteira reco...,20/08/2022 - 9:31,66.0,1
4,Mais Lidas,Respostas sobre Magazine Luiza (MGLU3) e mais ...,Investidores não tiraram Magazine Luiza (MGLU3...,28/08/2022 - 10:30,42.0,2


In [147]:
test.drop(['Categoria','Data','Pagina'], axis = 1, inplace = True)
test

,Titulo,PrimeiroParag,Target
0,"Ibovespa (IBOV) vira para queda, com VALE3 ent...",O Ibovespa (IBOV) operava sem forças nesta qua...,2
1,Ação da Braskem (BRKM3) avança com notícia sob...,As ações da Braskem (BRKM3) avançavam ao redor...,2
2,"Compre commodities, se preocupe com recessão d...",O Goldman Sachs recomenda que os investidores ...,1
3,Veja as 5 novas ações escaladas pela XP para e...,A XP Investimentos repaginou sua carteira reco...,1
4,Respostas sobre Magazine Luiza (MGLU3) e mais ...,Investidores não tiraram Magazine Luiza (MGLU3...,2
...,...,...,...
295,Ibovespa (IBOV) fecha em alta e recupera os 11...,O Ibovespa (IBOV) fechou em alta nesta quinta-...,1
296,"Ambev (ABEV3) está com desconto de 30%, calcul...",A Ambev (ABEV3) está com desconto de 30% sobre...,2
297,"brMalls (BRML3), Gerdau (GGBR4) e mais: Veja a...","Em atualização de carteira semanal, a XP Inves...",1
298,TSE homologa candidatura de Bolsonaro; Tribuna...,"Na sessão de julgamentos desta terça (6), os m...",0


In [148]:
train.drop(['Categoria','Data','Pagina'], axis = 1, inplace = True)
train

,Titulo,PrimeiroParag,Target
0,Ibovespa: ALPA4 fecha mais um pregão em alta e...,"O Ibovespa (IBOV) fechou em queda de 1,09% nes...",2
1,"IRB (IRBR3) é, de longe, a ação mais ‘shortead...",O IRB (IRBR3) é a ação com mais investidores s...,1
2,5 ações que pagam bons dividendos para investi...,"Para setembro, a Órama Investimentos optou por...",1
3,Empiricus Investimentos escala 6 ações com div...,"A Empiricus Investimentos, ex-Vitreo, incluiu ...",1
4,Ibovespa futuro opera em queda com aversão ao ...,O dólar abria o pregão desta segunda-feira (29...,0
...,...,...,...
445,“Vamos lançar um NFT nos próximos meses que va...,Neste sábado (3) aconteceu o ViralCure Festiva...,0
446,Digitalks terá presença de DAO 100% brasileira,"Cintia Ferreira, Fundadora da EVE e co-fundado...",0
447,"Vale (VALE3), Santander (SANB11) e mais 3 açõe...",A Ativa Investimentos retirou as ações de Bras...,2
448,Credores da chinesa Evergrande (EGRNF) montam ...,Fundos globais que investiram em títulos da ch...,1


In [149]:
# Nova variável
test['Texto Final']  = test['Titulo'] + ' ' + test['PrimeiroParag']
test

,Titulo,PrimeiroParag,Target,Texto Final
0,"Ibovespa (IBOV) vira para queda, com VALE3 ent...",O Ibovespa (IBOV) operava sem forças nesta qua...,2,"Ibovespa (IBOV) vira para queda, com VALE3 ent..."
1,Ação da Braskem (BRKM3) avança com notícia sob...,As ações da Braskem (BRKM3) avançavam ao redor...,2,Ação da Braskem (BRKM3) avança com notícia sob...
2,"Compre commodities, se preocupe com recessão d...",O Goldman Sachs recomenda que os investidores ...,1,"Compre commodities, se preocupe com recessão d..."
3,Veja as 5 novas ações escaladas pela XP para e...,A XP Investimentos repaginou sua carteira reco...,1,Veja as 5 novas ações escaladas pela XP para e...
4,Respostas sobre Magazine Luiza (MGLU3) e mais ...,Investidores não tiraram Magazine Luiza (MGLU3...,2,Respostas sobre Magazine Luiza (MGLU3) e mais ...
...,...,...,...,...
295,Ibovespa (IBOV) fecha em alta e recupera os 11...,O Ibovespa (IBOV) fechou em alta nesta quinta-...,1,Ibovespa (IBOV) fecha em alta e recupera os 11...
296,"Ambev (ABEV3) está com desconto de 30%, calcul...",A Ambev (ABEV3) está com desconto de 30% sobre...,2,"Ambev (ABEV3) está com desconto de 30%, calcul..."
297,"brMalls (BRML3), Gerdau (GGBR4) e mais: Veja a...","Em atualização de carteira semanal, a XP Inves...",1,"brMalls (BRML3), Gerdau (GGBR4) e mais: Veja a..."
298,TSE homologa candidatura de Bolsonaro; Tribuna...,"Na sessão de julgamentos desta terça (6), os m...",0,TSE homologa candidatura de Bolsonaro; Tribuna...


In [150]:
# Nova variável
train['Texto Final']  = train['Titulo'] + ' ' + train['PrimeiroParag']
train

,Titulo,PrimeiroParag,Target,Texto Final
0,Ibovespa: ALPA4 fecha mais um pregão em alta e...,"O Ibovespa (IBOV) fechou em queda de 1,09% nes...",2,Ibovespa: ALPA4 fecha mais um pregão em alta e...
1,"IRB (IRBR3) é, de longe, a ação mais ‘shortead...",O IRB (IRBR3) é a ação com mais investidores s...,1,"IRB (IRBR3) é, de longe, a ação mais ‘shortead..."
2,5 ações que pagam bons dividendos para investi...,"Para setembro, a Órama Investimentos optou por...",1,5 ações que pagam bons dividendos para investi...
3,Empiricus Investimentos escala 6 ações com div...,"A Empiricus Investimentos, ex-Vitreo, incluiu ...",1,Empiricus Investimentos escala 6 ações com div...
4,Ibovespa futuro opera em queda com aversão ao ...,O dólar abria o pregão desta segunda-feira (29...,0,Ibovespa futuro opera em queda com aversão ao ...
...,...,...,...,...
445,“Vamos lançar um NFT nos próximos meses que va...,Neste sábado (3) aconteceu o ViralCure Festiva...,0,“Vamos lançar um NFT nos próximos meses que va...
446,Digitalks terá presença de DAO 100% brasileira,"Cintia Ferreira, Fundadora da EVE e co-fundado...",0,Digitalks terá presença de DAO 100% brasileira...
447,"Vale (VALE3), Santander (SANB11) e mais 3 açõe...",A Ativa Investimentos retirou as ações de Bras...,2,"Vale (VALE3), Santander (SANB11) e mais 3 açõe..."
448,Credores da chinesa Evergrande (EGRNF) montam ...,Fundos globais que investiram em títulos da ch...,1,Credores da chinesa Evergrande (EGRNF) montam ...


In [151]:
target = list(train['Target'])

In [152]:
train0 = train.loc[train['Target'] == 0,:]
train0

,Titulo,PrimeiroParag,Target,Texto Final
4,Ibovespa futuro opera em queda com aversão ao ...,O dólar abria o pregão desta segunda-feira (29...,0,Ibovespa futuro opera em queda com aversão ao ...
8,Prisma Capital se torna titular de 56% da Domm...,"A Dommo Energia (DMMO3), antiga OGX de Eike Ba...",0,Prisma Capital se torna titular de 56% da Domm...
12,"Day Trade: Banco do Brasil (BBAS3), Marfrig (M...","A Ágora Investimentos, o BTG Pactual e o PagBa...",0,"Day Trade: Banco do Brasil (BBAS3), Marfrig (M..."
14,Diretores da Aneel aprovam novas tarifas de en...,A Agência Nacional de Energia Elétrica (Aneel)...,0,Diretores da Aneel aprovam novas tarifas de en...
15,"Bolsonaro no JN: centrão, pandemia, economia; ...","Durante 40 minutos, Jair Bolsonaro (PL) foi en...",0,"Bolsonaro no JN: centrão, pandemia, economia; ..."
...,...,...,...,...
439,Magalu (MGLU3) entra de cabeça no mundo dos ga...,O Magalu Games lançou seus primeiros três jogo...,0,Magalu (MGLU3) entra de cabeça no mundo dos ga...
441,"Depois das altas ligeiras, como na 5ª, vem a r...","A virada da soja e do milho ontem, na segunda ...",0,"Depois das altas ligeiras, como na 5ª, vem a r..."
444,Clima pós 7 de setembro: em meio às incertezas...,É fato consumado que vivemos as eleições mais ...,0,Clima pós 7 de setembro: em meio às incertezas...
445,“Vamos lançar um NFT nos próximos meses que va...,Neste sábado (3) aconteceu o ViralCure Festiva...,0,“Vamos lançar um NFT nos próximos meses que va...


In [153]:
train1 = train.loc[train['Target'] == 1,:]
train1

,Titulo,PrimeiroParag,Target,Texto Final
1,"IRB (IRBR3) é, de longe, a ação mais ‘shortead...",O IRB (IRBR3) é a ação com mais investidores s...,1,"IRB (IRBR3) é, de longe, a ação mais ‘shortead..."
2,5 ações que pagam bons dividendos para investi...,"Para setembro, a Órama Investimentos optou por...",1,5 ações que pagam bons dividendos para investi...
3,Empiricus Investimentos escala 6 ações com div...,"A Empiricus Investimentos, ex-Vitreo, incluiu ...",1,Empiricus Investimentos escala 6 ações com div...
6,BlackRock vê alta de ações de energia em merca...,O estrategista-chefe de investimentos da Black...,1,BlackRock vê alta de ações de energia em merca...
9,IRB (IRBR3) perde R$ 1 bilhão desde anúncio de...,O IRB (IRBR3) perdeu R$ 1 bilhão em valor de ...,1,IRB (IRBR3) perde R$ 1 bilhão desde anúncio de...
...,...,...,...,...
433,"Sem Wall Street, Ibovespa dispara puxado pelas...",O Ibovespa sobe forte na sessão desta segunda-...,1,"Sem Wall Street, Ibovespa dispara puxado pelas..."
435,"Day Trade: Petrobras (PETR4), Banco do Brasil ...","A Ágora Investimentos, o BTG Pactual, e o PagB...",1,"Day Trade: Petrobras (PETR4), Banco do Brasil ..."
437,Pechinchas da Bolsa e o que mais movimentou Co...,A Bolsa brasileira está barata. Isso não é nov...,1,Pechinchas da Bolsa e o que mais movimentou Co...
440,Day Trade: 9 ações para comprar hoje e buscar ...,"A Ágora Investimentos, o BTG Pactual, e o PagB...",1,Day Trade: 9 ações para comprar hoje e buscar ...


In [154]:
train2 = train.loc[train['Target'] == 2,:]
train2

,Titulo,PrimeiroParag,Target,Texto Final
0,Ibovespa: ALPA4 fecha mais um pregão em alta e...,"O Ibovespa (IBOV) fechou em queda de 1,09% nes...",2,Ibovespa: ALPA4 fecha mais um pregão em alta e...
5,Ibovespa pode se agarrar em menor cautela e na...,O Ibovespa (IBOV) seguiu ontem o mau humor das...,2,Ibovespa pode se agarrar em menor cautela e na...
7,"Apesar de dia mais fraco para o Ibovespa, CVCB...",O Ibovespa (IBOV) terminou o pregão fraco dest...,2,"Apesar de dia mais fraco para o Ibovespa, CVCB..."
10,"Small Caps têm rentabilidade 11,6% maior que o...","A rentabilidade anual das Small Caps foi 11,6%...",2,"Small Caps têm rentabilidade 11,6% maior que o..."
13,"IRB (IRBR3): Após a oferta de ações, é hora de...",O IRB (IRBR3) recuperou parte das perdas com a...,2,"IRB (IRBR3): Após a oferta de ações, é hora de..."
...,...,...,...,...
438,Petrobras (PETR4) sobe 2% mesmo com corte no p...,As ações da Petrobras (PETR3; PETR4) fecharam ...,2,Petrobras (PETR4) sobe 2% mesmo com corte no p...
442,"Banco do Brasil (BBAS3): Em novo preço-alvo, I...",O Itaú BBA elevou o preço-alvo para a ação do ...,2,"Banco do Brasil (BBAS3): Em novo preço-alvo, I..."
443,Oi (OIBR3) sobe 11% em uma semana e ultrapassa...,A ação da Oi (OIBR3) deu um repique nos último...,2,Oi (OIBR3) sobe 11% em uma semana e ultrapassa...
447,"Vale (VALE3), Santander (SANB11) e mais 3 açõe...",A Ativa Investimentos retirou as ações de Bras...,2,"Vale (VALE3), Santander (SANB11) e mais 3 açõe..."


### Descrição do Projeto 1

O Projeto 1 usará como fonte de dados o Portal do Money Times e o assunto das notícias será "ações".
As nossas variáveis Target serão: relevante(2), neutra(1) e irrelevante(0).

Para relevante será analisado se: menciona diretamente o aumento ou queda na porcentagem de uma ação.
Para neutro será considerado: não fala diretamente sobre as ações mas sim as pessoas que estão envolvidas com elas.
Para irrelevantes será considerado: que a notícia não se relaciona a ações.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [155]:
# Função que limpa e deixa todas as letras em minúsculas
import re 

stemmer = SnowballStemmer('portuguese')
def cleanup(text):
    clean = text.lower()
    clean = re.sub("@[A-Za-z0-9_]+","", clean)
    clean = re.sub("#[A-Za-z0-9_]+","", clean)
    clean = re.sub(r"http\S+", "", clean)
    clean = unidecode.unidecode(clean)
    clean = re.sub("[^a-z0-9]"," ", clean)
    clean = clean.replace("\n","")
    
    clean_2 = ''
    lista_das_palavras = clean.split()
    for palavra in lista_das_palavras:
        nova_palavra = stemmer.stem(palavra)
        clean_2 += nova_palavra + ' '
    clean = clean_2.rstrip()
    
    return clean

In [156]:
# train.Titulo.to_string(index = False)

In [157]:
trainstr = train.Titulo.to_string(index = False) + train.PrimeiroParag.to_string(index = False)
teststr = test.Titulo.to_string(index = False) + test.PrimeiroParag.to_string(index = False)

train_clean = cleanup(trainstr)
test_clean = cleanup(teststr)

train_list = train_clean.split()
test_list = test_clean.split()

#train_clean

In [158]:
test_textof = test['Texto Final'].to_string(index = False)
test_textof = cleanup(test_textof)
teste = test_textof.split()

# test_textof

In [159]:
# Limpando os Textos finais de treinos
train0_tflimpo = cleanup(train0['Texto Final'].to_string(index=False))
train1_tflimpo = cleanup(train1['Texto Final'].to_string(index=False))
train2_tflimpo = cleanup(train2['Texto Final'].to_string(index=False))

In [160]:
# Lista com as palavras dos treinos
lista_t0 = list(train0_tflimpo.split())
lista_t1 = list(train1_tflimpo.split())
lista_t2 = list(train2_tflimpo.split())
lista_total = lista_t0 + lista_t1 + lista_t2
# Lista com todas as palavras das três listas mas sem repetição
lista_total_sem_repeticao = set(lista_t0 + lista_t1 + lista_t2) 

In [161]:
# Frequência absoluta das palavras em cada lista
freq_absoluta_t0 = pd.Series(lista_t0).value_counts()
freq_absoluta_t1 = pd.Series(lista_t1).value_counts()
freq_absoluta_t2 = pd.Series(lista_t2).value_counts()

In [162]:
# Frequência relativa das palavras em cada lista
freq_relativa_t0 = pd.Series(lista_t0).value_counts(True)
freq_relativa_t1 = pd.Series(lista_t1).value_counts(True)
freq_relativa_t2 = pd.Series(lista_t2).value_counts(True)

In [163]:
# Probabilidade das relevâncias
Prob_0 = len(lista_t0)/len(lista_total)
Prob_1 = len(lista_t1)/len(lista_total)
Prob_2 = len(lista_t2)/len(lista_total)

In [164]:
def NaiveBayes(titulo_noticia):
    probT0 = 1
    probT1 = 1
    probT2 = 1
    l0 = []
    titulo_noticia = titulo_noticia.split()
    for palavra in titulo_noticia:
        if palavra in lista_t0:
            probT0 *= freq_relativa_t0[palavra] / freq_relativa_t0.sum()
        elif palavra not in lista_t0:
            probT0 *= 1 / (len(lista_t0) + len(lista_total))
        if palavra in lista_t1:
            probT1 *= freq_relativa_t1[palavra] / freq_relativa_t1.sum()
        elif palavra not in lista_t1:
            probT1 *= 1 / (len(lista_t1) + len(lista_total))
        if palavra in lista_t2:
            probT2 *= freq_relativa_t2[palavra] / freq_relativa_t2.sum()
        elif palavra not in lista_t2:
            probT2 *= 1 / (len(lista_t2) + len(lista_total))
        probT0 *= Prob_0
        probT1 *= Prob_1
        probT2 *= Prob_2
    if probT0 > probT1 and probT0 > probT2:
        return 0
    elif probT1 > probT0 and probT1 > probT2:
        return 1
    elif probT2 > probT1 and probT2 > probT0:
        return 2

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [165]:
test['Texto Final'] = cleanup(str(test['Texto Final']))

In [166]:
verificador = test.copy()

verificador['clean'] = verificador.Titulo.apply(cleanup)
verificador['comp'] = verificador.clean.apply(NaiveBayes)

verificador.head(5)

,Titulo,PrimeiroParag,Target,Texto Final,clean,comp
0,"Ibovespa (IBOV) vira para queda, com VALE3 ent...",O Ibovespa (IBOV) operava sem forças nesta qua...,2,0 ibovesp ibov vir par qued com vale3 ent 1 ac...,ibovesp ibov vir par qued com vale3 entre as m...,2
1,Ação da Braskem (BRKM3) avança com notícia sob...,As ações da Braskem (BRKM3) avançavam ao redor...,2,0 ibovesp ibov vir par qued com vale3 ent 1 ac...,aca da brask brkm3 avanc com notic sobr contro...,1
2,"Compre commodities, se preocupe com recessão d...",O Goldman Sachs recomenda que os investidores ...,1,0 ibovesp ibov vir par qued com vale3 ent 1 ac...,compr commoditi se preocup com recessa depo di...,1
3,Veja as 5 novas ações escaladas pela XP para e...,A XP Investimentos repaginou sua carteira reco...,1,0 ibovesp ibov vir par qued com vale3 ent 1 ac...,vej as 5 nov aco escal pel xp par enfrent o ib...,1
4,Respostas sobre Magazine Luiza (MGLU3) e mais ...,Investidores não tiraram Magazine Luiza (MGLU3...,2,0 ibovesp ibov vir par qued com vale3 ent 1 ac...,respost sobr magazin luiz mglu3 e mais destaqu...,2


In [167]:
verd0 = verificador.loc[(verificador.comp == 0) & (verificador.Target == 0)]
verd1 = verificador.loc[(verificador.comp == 1) & (verificador.Target == 1)]
verd2 = verificador.loc[(verificador.comp == 2) & (verificador.Target == 2)]

fals0 = verificador.loc[(verificador.comp == 0) & (verificador.Target != 0)]
fals1 = verificador.loc[(verificador.comp == 1) & (verificador.Target != 1)]
fals2 = verificador.loc[(verificador.comp == 2) & (verificador.Target != 2)]

print(f'O percentual de verdadeiros negativos foi de: {100 * (verd0.shape[0] / verificador.shape[0]):.2f}%')
print(f'O percentual de falsos negativos foi de: {100 * (fals0.shape[0] / verificador.shape[0]):.2f}%')
print('\n')
print(f'O percentual de verdadeiros neutros foi de: {100 * (verd1.shape[0] / verificador.shape[0]):.2f}%')
print(f'O percentual de falsos neutros foi de: {100 * (fals1.shape[0] / verificador.shape[0]):.2f}%')
print('\n')
print(f'O percentual de verdadeiros positivos foi de: {100 * (verd2.shape[0] / verificador.shape[0]):.2f}%')
print(f'O percentual de falsos positivos foi de: {100 * (fals2.shape[0] / verificador.shape[0]):.2f}%')
print('\n')
print(f'A acurácia do modelo foi de: {100 * (verd0 + verd1 + verd2).shape[0] / test.shape[0]:.2f}%')

O percentual de verdadeiros negativos foi de: 10.00%
O percentual de falsos negativos foi de: 1.33%


O percentual de verdadeiros neutros foi de: 19.33%
O percentual de falsos neutros foi de: 26.67%


O percentual de verdadeiros positivos foi de: 21.33%
O percentual de falsos positivos foi de: 21.33%


A acurácia do modelo foi de: 50.67%


___
### Concluindo

De acordo com o Teorema de Bayes o cálculo da probabilidade é dado por: $$P(A|B) = \frac{P(B|A) P(A)}{P(B)}$$  
Portanto, foi criada uma função ("NaiveBayes") que aplicasse tal raciocínio, a mesma proporcionou a aquisição dos novos valores de relevância de acordo com o algortimo do Classificador, cujos dados proporcionaram o cálculo dos percentuais de valores verdadeiros e falsos, além da acurácia.

O classificador automático obtido, que possui como propósito a classificação de notícias de acordo com a sua relevância sobre "ações", adquiriu uma acurácia de 56.67% quando utilizado com os dados da planilha Teste, um percentual de verdadeiros negativos de 10.00%, verdadeiros neutros de 19.33%, e de verdadeiros positivos de 21.33%.

Tal resultado se deve ao fato de que o classificador em questão, foi afetado pelos falsos positivos, visto que atribuem uma classificação improcedente ao classificador. Ademais, vale ressaltar que ao utilizar um database de menor escala, é esperado que suas classificações sejam mais arbitrárias, no entanto, se o mesmo fosse criado a partir de uma coleta de dados mais abundante, seus resultados seriam mais precisos e satisfatórios, dentro do universo probabilístico dos dados em questão. Com isso, sendo capaz de receber um output binário responsável por definí-los como relevantes, neutros ou ainda, irrelevantes. Ou seja, classificar de forma autônoma os dados em questão.

Entretanto, pode se admitir um funcionamento razoável ao próprio classificador, porém, não ideal, uma vez que "naive" (traduzido do inglês, ingênuo) supõe uma desconsideração das relações entre as palavras, permitindo oportunidades para classificações errôneas a respeito de figuras de linguagem comumente utilizadas em textos como sarcasmo, dupla negação, ironia e duplo sentido. Além disso, apesar de um aumento na base de dados propiciar um resultado mais próximo do ideal, tais questões que dizem respeito a gramática, não seriam sanadas.

Por conseguinte, tendo em consideração o funcionamento razoável do produto em questão, isto é, dentro do âmbito matemático, com acurácia acima de 50.00%, seria possível um aprimoramento das funções do classificador, permitindo relacionar certas empresas ou países ao aumento ou queda das ações, ou as ações que estão mais crescendo ou diminuindo no momento, o que potencializaria não apenas sua funcionalidade, mas como o aumento de interesse no classificador por partes de acionistas e empresas relacionadas a compra e vendas de ações. Todavia, tal melhoria seria apenas possível com o financiamento do projeto, visto que a implementação dessas demais funcionalidades demandariam tempo e um conhecimento mais profundo, além disso, um apoio monetário incentivaria o projeto de classificador automático a um desenvolvimento constante, buscando melhorar sua interpretação e classificação de dados. 

### Dupla negação e sarcasmo

#### Sarcasmo:

Como o próprio nome Naive Bayes sugere, o classificador não considere a associação das palavras, o que implica numa imprecisão durante a classificação de notícias que possuirem em seu conteúdo a figura de linguagem sarcasmo. De tal forma, o classificador atua seguindo uma lógica razoável no campo da matemática, mas incoerente gramaticalmente.

Perante o que foi exposto, um exemplo de uma frase sarcástica dentro do classificador seria:

In [116]:
frase_sarcastica = 'as açoes estao tao altas quanto o chao'
if NaiveBayes(frase_sarcastica)==1:
    print('A frase é neutra')
elif NaiveBayes(frase_sarcastica)==2:
    print('A frase é relevante')
else:
    print('A frase é irrelevante')

A frase é relevante


#### Dupla Negação:

Seguindo o mesmo raciocínio do item anterior abordado, uma frase que possui dupla negação seria também classificada incorretamente:

In [121]:
frase_duplo_sentido = 'açoes nao nao estao crescendo'
if NaiveBayes(frase_duplo_sentido)==1:
    print('A frase é neutra')
elif NaiveBayes(frase_sarcastica)==2:
    print('A frase é relevante')
else:
    print('A frase é irrelevante')

A frase é neutra


Conforme os exemplos acima é possível inferir, portanto, que as notícias de caráter sarcástico ou com dupla negação muito provavelmente serão classificados de maneira incorreta. Tal incoerência na classificação deve-se, sobretudo, à ingenuidade do classificador de Naive Bayes.

### Por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador?

Para um maior êxito na classificação de notícias, foi necessário inicialmente uma classificação manual, que levou em consideração os valores semânticos e gramáticos de cada notícia. No entanto, o classificador usou como base apenas os valores algébricos, desconsiderando a possibilidade, por exemplo, de notícias com sarcasmo, dupla negação ou sentido, logo, se fosse utilizado para alimentar a base de dados do Treinamento, ao invés de melhorar a acurácia, na verdade diminuiria-la, visto que estaria propagando um viés ainda maior. 
De tal forma,o incremento da base de Treinamento utilizando o classificador não seria possível, já que a presença de um certo grau de subjetividade na classificação das notícias é essencial na obtenção de u.

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**